# ChatOllama를 사용한 로컬 LLM 활용 가이드

이 노트북은 Ollama를 사용하여 로컬 환경에서 LLM을 구동하고 활용하는 방법을 다룹니다.

## 주요 내용
- Ollama 설치 및 설정
- 한국어 모델 (EEVE-Korean) 사용
- JSON 형식 출력 지원
- 멀티모달 모델 (LLaVA) 활용
- Modelfile을 통한 커스텀 모델 생성

## 1. Ollama 설치 및 설정

### 프로그램 설치
Ollama를 지원되는 플랫폼(Mac / Linux / Windows)에 다운로드하고 설치하세요.

- 설치주소: [https://ollama.com/](https://ollama.com/)

### 모델 다운로드

#### 허깅페이스 모델
허깅페이스(HuggingFace)에서 오픈모델을 다운로드 받습니다 (.gguf 확장자)

#### Ollama 제공 모델
`ollama pull <name-of-model>` 명령을 사용하여 사용 가능한 LLM 모델을 가져오세요.
- 예: `ollama pull gemma:7b`

아래의 경로에 모델의 기본 태그 버전이 다운로드됩니다.
- Mac: `~/.ollama/models`
- Linux/WSL: `/usr/share/ollama/.ollama/models`

### 주요 명령어
- `ollama list` - 가져온 모든 모델 확인
- `ollama run <name-of-model>` - 명령줄에서 모델과 직접 채팅
- `ollama pull <name-of-model>` - 모델 다운로드

## 2. 한국어 모델 사용하기

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_teddynote.messages import stream_response

# Ollama 한국어 모델을 불러옵니다.
llm = ChatOllama(model="EEVE-Korean-10.8B:latest")

# 프롬프트 생성
prompt = ChatPromptTemplate.from_template("{topic} 에 대하여 간략히 설명해 줘.")

# 체인 생성
chain = prompt | llm | StrOutputParser()

# 스트리밍 실행
answer = chain.stream({"topic": "deep learning"})

# 스트리밍 출력
stream_response(answer)

## 3. 비동기 스트리밍

In [ ]:
# 비동기적으로 체인을 실행하여 청크 단위로 결과를 반환
async for chunks in chain.astream(
    {"topic": "구글"}
):
    print(chunks, end="", flush=True)

## 4. JSON 형식 출력

일부 모델(예: gemma)은 JSON 형식 출력을 지원합니다.

In [ ]:
from langchain_ollama import ChatOllama

# JSON 형식 출력을 지원하는 모델 설정
llm = ChatOllama(
    model="gemma:7b",
    format="json",  # 입출력 형식을 JSON으로 설정
    temperature=0,
)

# JSON 형식의 답변을 요구하는 프롬프트
prompt = "유럽 여행지 10곳을 알려주세요. key: `places`. response in JSON format."

# 실행
response = llm.invoke(prompt)
print(response.content)

## 5. Modelfile을 통한 커스텀 모델 생성

### Modelfile 구성 요소
```
FROM [모델파일명]              # 사용할 GGUF 파일 지정
TEMPLATE """[프롬프트 템플릿]""" # 입출력 형식 정의
SYSTEM """[시스템 프롬프트]"""   # AI 역할 정의
PARAMETER [파라미터 설정]        # 추론 관련 설정
```

### 모델 생성 및 실행
```bash
# 모델 생성
ollama create [모델별칭] -f Modelfile

# 모델 실행
ollama run [모델별칭]
```

### 주요 파라미터
```
PARAMETER temperature 0      # 출력의 무작위성 (0: 결정적, 1: 무작위)
PARAMETER num_predict 3000   # 최대 토큰 생성 수
PARAMETER num_ctx 4096      # 컨텍스트 윈도우 크기
PARAMETER stop <s>          # 생성 중단 토큰
PARAMETER stop </s>         # 생성 중단 토큰
```

## 6. 멀티모달 모델 활용 (LLaVA)

Ollama는 bakllava와 llava와 같은 멀티모달 LLM을 지원합니다.

### 설치
```bash
ollama pull llava:7b
# 또는
ollama pull bakllava
```

### 이미지 처리를 위한 유틸리티 함수

In [ ]:
import base64
from io import BytesIO
from IPython.display import HTML, display
from PIL import Image
from langchain_core.messages import HumanMessage

def convert_to_base64(pil_image):
    """
    PIL 이미지를 Base64로 인코딩된 문자열로 변환합니다.
    """
    buffered = BytesIO()
    pil_image.save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

def plt_img_base64(img_base64):
    """
    Base64로 인코딩된 문자열을 이미지로 표시합니다.
    """
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'
    display(HTML(image_html))

def prompt_func(data):
    """
    이미지와 텍스트를 포함하는 프롬프트를 생성합니다.
    """
    text = data["text"]
    image = data["image"]

    image_part = {
        "type": "image_url",
        "image_url": f"data:image/jpeg;base64,{image}",
    }

    text_part = {"type": "text", "text": text}

    return [HumanMessage(content=[image_part, text_part])]

### 멀티모달 모델 사용 예제

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

# 이미지 로드 및 변환
file_path = "../image/jeju-beach.jpg"
pil_image = Image.open(file_path)
image_b64 = convert_to_base64(pil_image)

# 이미지 표시
plt_img_base64(image_b64)

# 멀티모달 모델 설정
llm = ChatOllama(model="llava:7b", temperature=0)

# 체인 생성
chain = prompt_func | llm | StrOutputParser()

# 이미지 설명 요청
result = chain.invoke(
    {"text": "Describe a picture in bullet points", "image": image_b64}
)

print(result)

## 7. 실제 활용시 주의사항

1. **파일 위치**: GGUF 파일과 Modelfile이 같은 디렉토리에 있어야 함
2. **파일명 정확성**: 파일명이 정확히 일치해야 함
3. **양자화 레벨**: Q4_0, Q8_0 등 양자화 레벨 확인 필요
4. **시스템 리소스**: RAM, GPU 고려 필요
5. **모델 크기**: 모델에 따라 필요한 메모리가 다름

### 권장 사양
- 7B 모델: 8GB RAM 이상
- 13B 모델: 16GB RAM 이상
- 70B 모델: 32GB RAM 이상 + GPU 권장

## 8. 요약

이 노트북에서는 다음 내용을 학습했습니다:

1. **Ollama 설치 및 설정** - 로컬 환경에서 LLM 구동
2. **한국어 모델 활용** - EEVE-Korean 모델 사용법
3. **JSON 출력** - 구조화된 데이터 생성
4. **커스텀 모델 생성** - Modelfile을 통한 설정
5. **멀티모달 모델** - 이미지와 텍스트 함께 처리

Ollama를 사용하면 클라우드 API 없이도 강력한 LLM을 로컬에서 활용할 수 있습니다.